# 1. Data Collection
- In this section, we are going to extract relevant dataset from 4 different data sources:
  - 1-1. Big Query Google Cloud Platofrm (GCP): Chicago Taxi Trips
  - 1-2. Wikipedia - Chicago Community Areas (Available at: https://en.wikipedia.org/wiki/Community_areas_in_Chicago)
  - 1-3. Nominatim API - OpenStreetMap Data (Avaiable at: https://nominatim.org/)
  - 1-4. Flatfile: taxi_vehicle.csv file sourced from Chicago Data Portable (Available at: https://data.cityofchicago.org/Community-Economic-Development/Active-Taxis-Make-Model-Chart/6cak-z3a4). 

## 1-1. Extracting Public Dataset from Google Cloud Platform
- Due to the large size of the dataset on GCP and the slow loading speed on our current local machine, this project will focus on trips that took place in 2015. It will specifically analyze the top 5 highest demand taxi companies: 'Yellow Cab', 'American United', 'Checker Taxi', 'Blue Diamond', and '5 Star Taxi'.

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()


QUERY = """
SELECT * 
FROM `chicago_taxi.chicago_taxi_main`
WHERE trip_year = 2015
AND company IN ('Yello Cab', 'American United', 'Checker Taxi', 'Blue Diamond', '5 Star Taxi')
"""

query_job = client.query(QUERY)

In [ ]:
chicago_taxi = query_job.to_dataframe()

In [ ]:
# chicago_taxi.to_csv("chicago_taxi.csv", index=False)

In [184]:
import pandas as pd
chicago_taxi = pd.read_csv("chicago_taxi.csv")

In [ ]:
chicago_taxi.info()

## 1-2. Webscraping Community Area Information

- In the Chicago Taxi Trip dataset from GCP, the community area is represented by numbers, suggesting that there could be additional valuable information to explore. This information might be particularly useful for implementing business strategies, such as when a taxi company targets specific areas and demographic groups. For future use, we will store this information in our database.


In [72]:
import requests
import re
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/Community_areas_in_Chicago'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

tbody = soup.find('tbody')
if tbody:
    rows = tbody.find_all('tr')

    community_numbers = []
    community_names = []
    populations = []

    for row in rows[1:]:  
        cols = row.find_all('td')
        if len(cols) > 2:
            community_numbers.append(cols[0].text.strip())
            community_names.append(row.find('th').text.strip()) 
            populations.append(cols[1].text.strip()) 

    community = pd.DataFrame({
        'community_number': community_numbers,
        'community_name': community_names,
        'population': populations
    })
    community=community[0:77]
    print(community)
else:
    print("Table body not found on the page")


   community_number      community_name population
0                01         Rogers Park     55,628
1                02          West Ridge     77,122
2                03              Uptown     57,182
3                04      Lincoln Square     40,494
4                05        North Center     35,114
..              ...                 ...        ...
72               73  Washington Heights     25,065
73               74     Mount Greenwood     18,628
74               75         Morgan Park     21,186
75               76              O'Hare     13,418
76               77           Edgewater     56,296

[77 rows x 3 columns]


## 1-3. Nominatim API - OpenStreetMap

- We are going to use the Nominatim API, which provides detailed geographic information, including specific addresses and types of locations based on specific geographic coordinates (i.e., latitude, longitude). As we have geo-coordinates provided in the Chicago taxi trips dataset, we are able to identify the pick-up and drop-off information in more detail.

In [ ]:
def get_location_details(lat, lon):
    url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        location_name = data.get('display_name')
        location_type = data.get('type')
        return pd.Series([location_name, location_type])
    else:
        return pd.Series(["Error", "Error"])


- To intergrade the API data, we first need to manipulate the chicago taxi trip dataset by concatenate the latitude and longitude to get the unique single location. Thus, we are going to create addition columns called 'pickup_location' and 'dropoff_location'.

In [3]:
chicago_taxi['pickup_location'] = chicago_taxi.apply(lambda row: f"{row['pickup_latitude']}, {row['pickup_longitude']}", axis=1)
chicago_taxi['dropoff_location'] = chicago_taxi.apply(lambda row: f"{row['dropoff_latitude']}, {row['dropoff_longitude']}", axis=1)

- We are going to create two tables, named 'pickup_location_info' and 'dropoff_location_info', to integrate information from the Nominatim API for the extraction of additional geographic information, respectively. Each table will contain the following information: pickup_location, latitude, longitude, address, and location_type.

In [ ]:

# creating pikup location table intergrating API
pickup_location = chicago_taxi['pickup_location'].unique()
pickup_location_info = pd.DataFrame(pickup_location)
pickup_location_info.rename(columns={0: 'pickup_location'}, inplace=True)
pickup_location_info[['latitude', 'longitude']] = pickup_location_info['pickup_location'].str.split(',', expand=True)
pickup_location_info[['address','type']] = pickup_location_info.apply(lambda row: get_location_details(row['latitude'], row['longitude']), axis=1)
pickup_location_info['latitude'] = pickup_location_info['latitude'].astype(float)
pickup_location_info['longitude'] = pickup_location_info['longitude'].astype(float)

# creating dropoff location table intergrating API
dropoff_location = chicago_taxi['dropoff_location'].unique()
dropoff_location_info = pd.DataFrame(dropoff_location)
dropoff_location_info.rename(columns={0: 'dropoff_location'}, inplace=True)
dropoff_location_info[['latitude', 'longitude']] = dropoff_location_info['dropoff_location'].str.split(',', expand=True)
dropoff_location_info[['address','type']] = dropoff_location_info.apply(lambda row: get_location_details(row['latitude'], row['longitude']), axis=1)
dropoff_location_info['latitude'] = dropoff_location_info['latitude'].astype(float)
dropoff_location_info['longitude'] = dropoff_location_info['longitude'].astype(float)



In [ ]:
# display(pickup_location_info.head())
# display(dropoff_location_info.head())

In [54]:
# merge dropoff and pickup location tables
dropoff_location_info.rename(columns={'dropoff_location': 'location_coordinates', 'type': 'dropoff_type'}, inplace=True)
pickup_location_info.rename(columns={'pickup_location': 'location_coordinates', 'type': 'pickup_type'}, inplace=True)

location_info = pd.merge(dropoff_location_info, pickup_location_info, on='location_coordinates', suffixes=('_dropoff', '_pickup'))

location_info = location_info[['location_coordinates', 'address_dropoff', 'dropoff_type']]
location_info.rename(columns={'dropoff_type': 'type'}, inplace=True)
location_info.rename(columns={'address_dropoff': 'address'}, inplace=True)


In [76]:
location_info = pd.read_csv("location_info.csv")

# 1-4. Flatfile: Taxi Vehicle type/make, Taxi Comapny Info
- **Taxi Vehicle type/make**: This contains the information of type and make of taxis running in Chicago. The presented dataset isn't exactaly same as from original source. I've modified to make it relevent to the dataset from other existing data sources to meet the RNCP criteria (i.e. randomly assign taxi_id)
- **Taxi company**: the data sourced as flatfiles from chicago city data portal - only 2020 data available. comapny_id is assgined to be stored as primary key in the database

In [91]:
taxi = pd.read_csv('taxi_vehicle.csv')
taxi.head()

,taxi_id,Public Vehicle Number,Vehicle Make,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source
0,6e40306a3a76d2e41f2530cc314ecfbd2520aae13202d7...,1350,FORD,2014.0,BLUE,Hybrid
1,6aeb4a88ff55ac575e3ef10ef32622967534bd48fb9ba6...,4063,NISSAN,2011.0,WHITE,Hybrid
2,641c9356c873f4b5fb13d4b2f70d8b4d4b7b2c98057272...,5448,CHRYSLER,2013.0,YELLOW,Flex Fuel
3,8307cf9433f0293eee99c6944aeab484521d9cd9b1fce5...,266,TOYOTA,2013.0,GREEN,Hybrid
4,687e3ef9daf087b79188bf0fea27f22cd5786b0cda0c80...,5644,TOYOTA,2012.0,WHITE,Hybrid


In [93]:
taxi_comapny_id = pd.read_csv("taxi_company1.csv")
taxi_comapny_info = pd.read_csv("taxi_company2.csv")



In [94]:
company = pd.merge(taxi_comapny_info, taxi_comapny_id, on='company', how='left')
company=company[['company_id','company', 'taxi_exterior_color', 'business_phone', 'dispatch_phone',
       'address', 'city_state', 'zip', 'email']]

In [95]:
company.head()

,company_id,company,taxi_exterior_color,business_phone,dispatch_phone,address,city_state,zip,email
0,22,5 Star Taxi,White,773-561-4444,773-561-4444,9696 W. FOSTER AVE,"CHICAGO, IL",60656,info@flash.com
1,16,24 Seven Taxi,Blue,773-878-8294,773-944-0350,5606 N. WESTERN AV,"CHICAGO, IL",60659,chicago247taxi@gmail.com
2,4,American United,"White, Stars, Stripes",773-327-6161,773-248-7600,"3800 N MILWAUKEE AVE, SUITE A","CHICAGO, IL",60641,NaN
3,12,Blue Diamond,"Cream, Blue",312-881-3188,312-226-8880,"3800 N MILWAUKEE AVE, SUITE A","CHICAGO, IL",60641,NaN
4,21,Blue Ribbon Taxi Association Inc.,"White, Blue, Stripes",773-279-4100,773-878-5400,4020 W. GLENLAKE AVE,"CHICAGO, IL",60646,info@blueribbontaxi.com


# 2. Data Cleaning

- We are going to conduct simple data cleaning for storing into database. We are going to convert the datatypes correctly and remove the columns that might be considered redundant. Further, we will manipulate the tables according to optimization to database (e.g. primary and foreign keys)

In [127]:
chicago_taxi.head()

,unique_key,taxi_id,trip_year,trip_start_timestamp,trip_start_date,trip_start_time,trip_end_timestamp,trip_end_date,trip_end_time,trip_seconds,...,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,3b838244692ab501427001b44af93f77c86204d4,6e40306a3a76d2e41f2530cc314ecfbd2520aae13202d7...,2015,2015-04-19 00:00:00+00:00,2015-04-19,00:00:00,2015-04-19 00:15:00+00:00,2015-04-19,00:15:00,147.0,...,2.0,NaN,1.0,7.05,Credit Card,Blue Diamond,41.942585,-87.656644,41.942692,-87.651771
1,261244b20b7cc4bb971b9d21bc9f756bd8ee07d9,6aeb4a88ff55ac575e3ef10ef32622967534bd48fb9ba6...,2015,2015-04-19 00:00:00+00:00,2015-04-19,00:00:00,2015-04-19 00:00:00+00:00,2015-04-19,00:00:00,552.0,...,1.0,NaN,1.5,10.95,Credit Card,American United,41.885281,-87.657233,41.912432,-87.670189
2,eac1e0162828f4ddd7ae137adb168c0831a3bf72,641c9356c873f4b5fb13d4b2f70d8b4d4b7b2c98057272...,2015,2015-04-19 00:00:00+00:00,2015-04-19,00:00:00,2015-04-19 00:15:00+00:00,2015-04-19,00:15:00,629.0,...,3.0,NaN,1.5,11.15,Credit Card,American United,41.893216,-87.637844,41.892042,-87.631864
3,5f54f3b346e021ac2549229752cb6f59f6065653,8307cf9433f0293eee99c6944aeab484521d9cd9b1fce5...,2015,2015-04-19 00:00:00+00:00,2015-04-19,00:00:00,2015-04-19 00:00:00+00:00,2015-04-19,00:00:00,182.0,...,0.0,NaN,1.0,5.65,Cash,American United,41.936237,-87.656412,41.941556,-87.666289
4,3fa6dc0f02a81ee04349f398fdbc0f583c1669ff,687e3ef9daf087b79188bf0fea27f22cd5786b0cda0c80...,2015,2015-04-19 00:00:00+00:00,2015-04-19,00:00:00,2015-04-19 00:15:00+00:00,2015-04-19,00:15:00,708.0,...,0.0,NaN,1.5,10.95,Cash,American United,41.895033,-87.619711,41.892493,-87.664746


In [128]:
chicago_taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1778219 entries, 0 to 1778218
Data columns (total 24 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   unique_key              object 
 1   taxi_id                 object 
 2   trip_year               int64  
 3   trip_start_timestamp    object 
 4   trip_start_date         object 
 5   trip_start_time         object 
 6   trip_end_timestamp      object 
 7   trip_end_date           object 
 8   trip_end_time           object 
 9   trip_seconds            float64
 10  trip_miles              float64
 11  pickup_community_area   float64
 12  dropoff_community_area  float64
 13  fare                    float64
 14  tips                    float64
 15  tolls                   float64
 16  extras                  float64
 17  trip_total              float64
 18  payment_type            object 
 19  company                 object 
 20  pickup_latitude         float64
 21  pickup_longitude        float64

In [129]:
chicago_taxi.duplicated().sum()

0

In [185]:
chicago_taxi = chicago_taxi.drop(['trip_year','trip_start_date','trip_start_time','trip_end_date','trip_end_time'], axis=1)

chicago_taxi['trip_start_timestamp'] = pd.to_datetime(chicago_taxi['trip_start_timestamp']).dt.strftime('%Y-%m-%d %H:%M:%S')
chicago_taxi['trip_end_timestamp'] = pd.to_datetime(chicago_taxi['trip_end_timestamp']).dt.strftime('%Y-%m-%d %H:%M:%S')

chicago_taxi['pickup_community_area'] = chicago_taxi['pickup_community_area'].astype('Int64')
chicago_taxi['dropoff_community_area'] = chicago_taxi['dropoff_community_area'].astype('Int64')

In [186]:
chicago_taxi = pd.merge(chicago_taxi, taxi_comapny_id, on='company', how='left')

In [187]:
chicago_taxi = chicago_taxi.drop(['company'], axis=1)

In [188]:
chicago_taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1778219 entries, 0 to 1778218
Data columns (total 19 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   unique_key              object 
 1   taxi_id                 object 
 2   trip_start_timestamp    object 
 3   trip_end_timestamp      object 
 4   trip_seconds            float64
 5   trip_miles              float64
 6   pickup_community_area   Int64  
 7   dropoff_community_area  Int64  
 8   fare                    float64
 9   tips                    float64
 10  tolls                   float64
 11  extras                  float64
 12  trip_total              float64
 13  payment_type            object 
 14  pickup_latitude         float64
 15  pickup_longitude        float64
 16  dropoff_latitude        float64
 17  dropoff_longitude       float64
 18  company_id              int64  
dtypes: Int64(2), float64(11), int64(1), object(5)
memory usage: 261.2+ MB


In [42]:
chicago_taxi.isnull().sum()

unique_key                      0
taxi_id                         0
trip_start_timestamp            0
trip_end_timestamp              0
trip_seconds                   85
trip_miles                      0
pickup_community_area       46967
dropoff_community_area      81578
fare                            0
tips                            0
tolls                     1778219
extras                          0
trip_total                      0
payment_type                    0
company                         0
pickup_latitude             46885
pickup_longitude            46885
dropoff_latitude            78108
dropoff_longitude           78108
dtype: int64

In [209]:
trips=chicago_taxi.copy()

In [210]:
# Example for the 'trips' DataFrame
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1778219 entries, 0 to 1778218
Data columns (total 19 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   unique_key              object 
 1   taxi_id                 object 
 2   trip_start_timestamp    object 
 3   trip_end_timestamp      object 
 4   trip_seconds            float64
 5   trip_miles              float64
 6   pickup_community_area   Int64  
 7   dropoff_community_area  Int64  
 8   fare                    float64
 9   tips                    float64
 10  tolls                   float64
 11  extras                  float64
 12  trip_total              float64
 13  payment_type            object 
 14  pickup_latitude         float64
 15  pickup_longitude        float64
 16  dropoff_latitude        float64
 17  dropoff_longitude       float64
 18  company_id              int64  
dtypes: Int64(2), float64(11), int64(1), object(5)
memory usage: 261.2+ MB


In [212]:
import numpy as np

# Example for handling 'trips_df'

# Convert nullable integers and booleans to None
int_bool_cols = trips.select_dtypes(include=['Int64', 'boolean']).columns
trips[int_bool_cols] = trips[int_bool_cols].where(trips[int_bool_cols].notna(), None)

# Convert nullable timestamps to None
timestamp_cols = ['trip_start_timestamp', 'trip_end_timestamp']  # Add other timestamp columns if present
for col in timestamp_cols:
    trips[col] = pd.to_datetime(trips[col], errors='coerce')
    trips[col] = trips[col].where(trips[col].notna(), None)

# Handle strings (if your schema allows NULLs in these columns)
string_cols = trips.select_dtypes(include=['object']).columns
trips[string_cols] = trips[string_cols].where(trips[string_cols].notna(), None)
# If your schema does not allow NULLs, use .fillna('') instead of .where(..., None)


In [45]:
community

,community_number,community_name,population
0,01,Rogers Park,"55,628"
1,02,West Ridge,"77,122"
2,03,Uptown,"57,182"
3,04,Lincoln Square,"40,494"
4,05,North Center,"35,114"
...,...,...,...
72,73,Washington Heights,"25,065"
73,74,Mount Greenwood,"18,628"
74,75,Morgan Park,"21,186"
75,76,O'Hare,"13,418"


In [73]:
community['community_number'] = community['community_number'].astype('Int64')
community['population'] = community['population'].str.replace(',','').astype('Int64')

In [74]:
community.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   community_number  77 non-null     Int64 
 1   community_name    77 non-null     object
 2   population        77 non-null     Int64 
dtypes: Int64(2), object(1)
memory usage: 2.1+ KB


In [78]:
community.isnull().sum()

community_number    0
community_name      0
population          0
dtype: int64

In [77]:
location_info

,location_coordinates,address,type
0,"41.942691844, -87.651770507","860-862, West Buckingham Place, Northalsted, L...",yes
1,"41.912431869, -87.670189148","1707-1709, North Paulina Street, Wicker Park, ...",yes
2,"41.892042136, -87.63186395","Best Western Plus River North, 125, West Ohio ...",hotel
3,"41.941555829, -87.666288887","1501, West School Street, Lake View, Chicago, ...",yes
4,"41.892493167, -87.664745836","1462, West Ohio Street, West Town, Chicago, We...",yes
...,...,...,...
416,"41.95679809, -87.772043822","4200, North Menard Avenue, Martin Luther, Port...",yes
417,"41.980843616, -87.736083927","5455, North Kildare Avenue, North Park, Chicag...",yes
418,"41.782113893, -87.739958858","4698, West 61st Street, Clearing, Chicago, Lak...",house
419,"41.827613139, -87.604241624","3753, South Lake Park Avenue, Oakland, Chicago...",yes


In [79]:
location_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   location_coordinates  421 non-null    object
 1   address               421 non-null    object
 2   type                  421 non-null    object
dtypes: object(3)
memory usage: 10.0+ KB


In [81]:
location_info.isnull().sum()

location_coordinates    0
address                 0
type                    0
dtype: int64

In [117]:
location = location_info.copy()

'type' of address appear to be not informative as most of values are yes - which does not mean anythin. So we drop this column

In [84]:
location_info['type'].value_counts()

type
yes                 251
apartments           31
house                22
residential          14
parking              12
school               12
bridge                5
university            5
secondary             5
motorway              4
pitch                 4
golf_course           3
bus_stop              3
nature_reserve        3
industrial            3
brewery               2
garden                2
aerodrome             2
playground            2
terrace               2
church                2
office                2
restaurant            2
detached              2
zoo                   1
surveillance          1
path                  1
theatre               1
bar                   1
post_depot            1
college               1
tertiary              1
museum                1
pub                   1
Error                 1
service               1
dog_park              1
department_store      1
hospital              1
religious             1
sports_centre         1
recycling  

In [85]:
location_info = location_info.drop(columns=['type'], axis=1)

In [98]:
company.head()

,company_id,company,taxi_exterior_color,business_phone,dispatch_phone,address,city_state,zip,email
0,22,5 Star Taxi,White,773-561-4444,773-561-4444,9696 W. FOSTER AVE,"CHICAGO, IL",60656,info@flash.com
1,16,24 Seven Taxi,Blue,773-878-8294,773-944-0350,5606 N. WESTERN AV,"CHICAGO, IL",60659,chicago247taxi@gmail.com
2,4,American United,"White, Stars, Stripes",773-327-6161,773-248-7600,"3800 N MILWAUKEE AVE, SUITE A","CHICAGO, IL",60641,NaN
3,12,Blue Diamond,"Cream, Blue",312-881-3188,312-226-8880,"3800 N MILWAUKEE AVE, SUITE A","CHICAGO, IL",60641,NaN
4,21,Blue Ribbon Taxi Association Inc.,"White, Blue, Stripes",773-279-4100,773-878-5400,4020 W. GLENLAKE AVE,"CHICAGO, IL",60646,info@blueribbontaxi.com


In [99]:
company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   company_id           19 non-null     int64 
 1   company              19 non-null     object
 2   taxi_exterior_color  19 non-null     object
 3   business_phone       19 non-null     object
 4   dispatch_phone       19 non-null     object
 5   address              19 non-null     object
 6   city_state           19 non-null     object
 7   zip                  19 non-null     int64 
 8   email                13 non-null     object
dtypes: int64(2), object(7)
memory usage: 1.5+ KB


In [101]:
taxi.head()

,taxi_id,Public Vehicle Number,Vehicle Make,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source
0,6e40306a3a76d2e41f2530cc314ecfbd2520aae13202d7...,1350,FORD,2014.0,BLUE,Hybrid
1,6aeb4a88ff55ac575e3ef10ef32622967534bd48fb9ba6...,4063,NISSAN,2011.0,WHITE,Hybrid
2,641c9356c873f4b5fb13d4b2f70d8b4d4b7b2c98057272...,5448,CHRYSLER,2013.0,YELLOW,Flex Fuel
3,8307cf9433f0293eee99c6944aeab484521d9cd9b1fce5...,266,TOYOTA,2013.0,GREEN,Hybrid
4,687e3ef9daf087b79188bf0fea27f22cd5786b0cda0c80...,5644,TOYOTA,2012.0,WHITE,Hybrid


In [102]:
taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   taxi_id                386 non-null    object 
 1   Public Vehicle Number  386 non-null    int64  
 2   Vehicle Make           386 non-null    object 
 3   Vehicle Model Year     386 non-null    float64
 4   Vehicle Color          386 non-null    object 
 5   Vehicle Fuel Source    386 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 18.2+ KB


In [104]:
taxi.columns = taxi.columns.str.lower()
taxi.columns = taxi.columns.str.replace(' ', '_')

In [106]:
taxi['vehicle_model_year'] = taxi['vehicle_model_year'].astype('Int64')

# 3. Storing into Database

In [141]:
display(trips.head())
display(taxi.head())
display(location.head())
display(community.head())
display(company.head())

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,company_id
0,3b838244692ab501427001b44af93f77c86204d4,6e40306a3a76d2e41f2530cc314ecfbd2520aae13202d7...,2015-04-19 00:00:00+00:00,2015-04-19 00:15:00+00:00,147.0,0.2,6,6,4.05,2.0,NaN,1.0,7.05,Credit Card,41.942585,-87.656644,41.942692,-87.651771,12
1,261244b20b7cc4bb971b9d21bc9f756bd8ee07d9,6aeb4a88ff55ac575e3ef10ef32622967534bd48fb9ba6...,2015-04-19 00:00:00+00:00,2015-04-19 00:00:00+00:00,552.0,2.5,28,24,8.45,1.0,NaN,1.5,10.95,Credit Card,41.885281,-87.657233,41.912432,-87.670189,4
2,eac1e0162828f4ddd7ae137adb168c0831a3bf72,641c9356c873f4b5fb13d4b2f70d8b4d4b7b2c98057272...,2015-04-19 00:00:00+00:00,2015-04-19 00:15:00+00:00,629.0,0.5,8,8,6.65,3.0,NaN,1.5,11.15,Credit Card,41.893216,-87.637844,41.892042,-87.631864,4
3,5f54f3b346e021ac2549229752cb6f59f6065653,8307cf9433f0293eee99c6944aeab484521d9cd9b1fce5...,2015-04-19 00:00:00+00:00,2015-04-19 00:00:00+00:00,182.0,0.8,6,6,4.65,0.0,NaN,1.0,5.65,Cash,41.936237,-87.656412,41.941556,-87.666289,4
4,3fa6dc0f02a81ee04349f398fdbc0f583c1669ff,687e3ef9daf087b79188bf0fea27f22cd5786b0cda0c80...,2015-04-19 00:00:00+00:00,2015-04-19 00:15:00+00:00,708.0,2.6,8,24,9.45,0.0,NaN,1.5,10.95,Cash,41.895033,-87.619711,41.892493,-87.664746,4


,taxi_id,public_vehicle_number,vehicle_make,vehicle_model_year,vehicle_color,vehicle_fuel_source
0,6e40306a3a76d2e41f2530cc314ecfbd2520aae13202d7...,1350,FORD,2014,BLUE,Hybrid
1,6aeb4a88ff55ac575e3ef10ef32622967534bd48fb9ba6...,4063,NISSAN,2011,WHITE,Hybrid
2,641c9356c873f4b5fb13d4b2f70d8b4d4b7b2c98057272...,5448,CHRYSLER,2013,YELLOW,Flex Fuel
3,8307cf9433f0293eee99c6944aeab484521d9cd9b1fce5...,266,TOYOTA,2013,GREEN,Hybrid
4,687e3ef9daf087b79188bf0fea27f22cd5786b0cda0c80...,5644,TOYOTA,2012,WHITE,Hybrid


,location_coordinates,address
0,"41.942691844, -87.651770507","860-862, West Buckingham Place, Northalsted, L..."
1,"41.912431869, -87.670189148","1707-1709, North Paulina Street, Wicker Park, ..."
2,"41.892042136, -87.63186395","Best Western Plus River North, 125, West Ohio ..."
3,"41.941555829, -87.666288887","1501, West School Street, Lake View, Chicago, ..."
4,"41.892493167, -87.664745836","1462, West Ohio Street, West Town, Chicago, We..."


,community_number,community_name,population
0,1,Rogers Park,55628
1,2,West Ridge,77122
2,3,Uptown,57182
3,4,Lincoln Square,40494
4,5,North Center,35114


,company_id,company,taxi_exterior_color,business_phone,dispatch_phone,address,city_state,zip,email
0,22,5 Star Taxi,White,773-561-4444,773-561-4444,9696 W. FOSTER AVE,"CHICAGO, IL",60656,info@flash.com
1,16,24 Seven Taxi,Blue,773-878-8294,773-944-0350,5606 N. WESTERN AV,"CHICAGO, IL",60659,chicago247taxi@gmail.com
2,4,American United,"White, Stars, Stripes",773-327-6161,773-248-7600,"3800 N MILWAUKEE AVE, SUITE A","CHICAGO, IL",60641,NaN
3,12,Blue Diamond,"Cream, Blue",312-881-3188,312-226-8880,"3800 N MILWAUKEE AVE, SUITE A","CHICAGO, IL",60641,NaN
4,21,Blue Ribbon Taxi Association Inc.,"White, Blue, Stripes",773-279-4100,773-878-5400,4020 W. GLENLAKE AVE,"CHICAGO, IL",60646,info@blueribbontaxi.com


In [157]:
display(location.dtypes)
# display(taxi.dtypes)
# display(community.dtypes)
# display(company.dtypes)
# display(location.dtypes)



location_coordinates    object
address                 object
dtype: object

In [213]:
import mysql.connector
from mysql.connector import Error
import pandas as pd


hostname = '127.0.0.1'
port = 3306
dbname = 'chicago_taxi'
username = 'root'
password = 'password'

connection = None

try:
    # Establish connection to the MySQL database
    connection = mysql.connector.connect(host=hostname, port=port, database=dbname, user=username, password=password)

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()

        # Create tables
        create_tables_queries = {
            'trips': """
                CREATE TABLE IF NOT EXISTS trips (
                    unique_key VARCHAR(255),
                    taxi_id VARCHAR(255),
                    trip_start_timestamp DATETIME,
                    trip_end_timestamp DATETIME,
                    trip_seconds FLOAT,
                    trip_miles FLOAT,
                    pickup_community_area INT,
                    dropoff_community_area INT,
                    fare FLOAT,
                    tips FLOAT,
                    tolls FLOAT,
                    extras FLOAT,
                    trip_total FLOAT,
                    payment_type VARCHAR(255),
                    pickup_latitude FLOAT,
                    pickup_longitude FLOAT,
                    dropoff_latitude FLOAT,
                    dropoff_longitude FLOAT,
                    company_id INT
                );
            """,
            'taxi': """
                CREATE TABLE IF NOT EXISTS taxi (
                    taxi_id VARCHAR(255),
                    public_vehicle_number INT,
                    vehicle_make VARCHAR(255),
                    vehicle_model_year INT,
                    vehicle_color VARCHAR(255),
                    vehicle_fuel_source VARCHAR(255)
                );
            """,
            'community': """
                CREATE TABLE IF NOT EXISTS community (
                    community_number INT,
                    community_name VARCHAR(255),
                    population INT
                );
            """,
            'company': """
                CREATE TABLE IF NOT EXISTS company (
                    company_id INT,
                    company VARCHAR(255),
                    taxi_exterior_color VARCHAR(255),
                    business_phone VARCHAR(255),
                    dispatch_phone VARCHAR(255),
                    address VARCHAR(255),
                    city_state VARCHAR(255),
                    zip INT,
                    email VARCHAR(255)
                );
            """,
            'location': """
                CREATE TABLE IF NOT EXISTS location (
                    location_coordinates VARCHAR(255),
                    address VARCHAR(255)
                );
            """
        }

        for table_name, create_query in create_tables_queries.items():
            cursor.execute(create_query)
            print(f"Table '{table_name}' created successfully")

        # Insert data into tables
        def insert_data(table_name, dataframe, insert_query):
            for i, row in dataframe.iterrows():
                cursor.execute(insert_query, tuple(row))
            connection.commit()
            print(f"Data inserted successfully into {table_name}")

       # Inserting data into the 'taxi' table
        taxi_insert_query = """
            INSERT INTO taxi (taxi_id, public_vehicle_number, vehicle_make, vehicle_model_year, vehicle_color, vehicle_fuel_source)
            VALUES (%s, %s, %s, %s, %s, %s)
        """
        insert_data('taxi', taxi, taxi_insert_query)

        # Inserting data into the 'trips' table
        trips_insert_query = """
            INSERT INTO trips (unique_key, taxi_id, trip_start_timestamp, trip_end_timestamp, trip_seconds, trip_miles, pickup_community_area, dropoff_community_area, fare, tips, tolls, extras, trip_total, payment_type, pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude, company_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        insert_data('trips', trips, trips_insert_query)

        # Inserting data into the 'community' table
        community_insert_query = """
            INSERT INTO community (community_number, community_name, population)
            VALUES (%s, %s, %s)
        """
        insert_data('community', community, community_insert_query)

        # Inserting data into the 'company' table
        company_insert_query = """
            INSERT INTO company (company_id, company, taxi_exterior_color, business_phone, dispatch_phone, address, city_state, zip, email)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        insert_data('company', company, company_insert_query)

        # Inserting data into the 'location' table
        location_insert_query = """
            INSERT INTO location (location_coordinates, address)
            VALUES (%s, %s)
        """
        insert_data('location', location, location_insert_query)


except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    # Close the connection if it was established
    if connection and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


Connected to MySQL Server version  8.0.33
Table 'trips' created successfully
Table 'taxi' created successfully
Table 'community' created successfully
Table 'company' created successfully
Table 'location' created successfully
Data inserted successfully into taxi
Error while connecting to MySQL Failed processing format-parameters; Python 'timestamp' cannot be converted to a MySQL type
MySQL connection is closed
